In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\kche8\Desktop\Dataset\bank.csv")
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
5,42,management,single,tertiary,no,0,yes,yes,unknown,5,may,562,2,-1,0,unknown,yes
6,56,management,married,tertiary,no,830,yes,yes,unknown,6,may,1201,1,-1,0,unknown,yes
7,60,retired,divorced,secondary,no,545,yes,no,unknown,6,may,1030,1,-1,0,unknown,yes
8,37,technician,married,secondary,no,1,yes,no,unknown,6,may,608,1,-1,0,unknown,yes
9,28,services,single,secondary,no,5090,yes,no,unknown,6,may,1297,3,-1,0,unknown,yes


In [2]:
df.shape

(11162, 17)

In [3]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [4]:
# Basic clean
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
)


In [5]:
# Replace literal 'unknown' with real missing values
df = df.replace('unknown', np.nan)

In [6]:
# Strip/standardize all text fields
for col in df.select_dtypes(include='object'):
    df[col] = df[col].astype(str).str.strip().str.lower()

In [7]:
#Convert numeric fields
numeric_cols = ['age','balance','day','duration','campaign','pdays','previous']
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

In [8]:
#Build proper date column (day + month + fixed year)
if {'day','month'}.issubset(df.columns):
    month_map = {
        'jan': '01', 'january': '01',
        'feb': '02', 'february': '02',
        'mar': '03', 'march': '03',
        'apr': '04', 'april': '04',
        'may': '05',
        'jun': '06', 'june': '06',
        'jul': '07', 'july': '07',
        'aug': '08', 'august': '08',
        'sep': '09', 'september': '09',
        'oct': '10', 'october': '10',
        'nov': '11', 'november': '11',
        'dec': '12', 'december': '12'
    }

    df['day_str'] = df['day'].fillna(1).astype(int).astype(str).str.zfill(2)
    df['month_num'] = df['month'].str.lower().map(month_map)

    df['date'] = pd.to_datetime(
        df['day_str'] + '-' + df['month_num'] + '-2014',
        format='%d-%m-%Y', errors='coerce'
    )

    df.drop(columns=['day_str','month_num'], inplace=True)

In [9]:
#Feature engineering for operations
if 'deposit' in df.columns:
    df['converted'] = np.where(df['deposit'].eq('yes'), 1, 0)

if 'campaign' in df.columns:
    df['is_repeat_contact'] = np.where(df['campaign'] > 1, 1, 0)

if 'pdays' in df.columns:
    df['is_followup'] = np.where(df['pdays'] >= 0, 1, 0)

if 'previous' in df.columns:
    df['had_prior_contact'] = np.where(df['previous'] > 0, 1, 0)

if 'duration' in df.columns:
    df['call_seconds'] = df['duration'].clip(lower=0)

if 'date' in df.columns:
    df['contact_year']  = df['date'].dt.year
    df['contact_month'] = df['date'].dt.to_period('M').astype(str)
    df['contact_week']  = df['date'].dt.to_period('W').astype(str)
    df['contact_dow']   = df['date'].dt.day_name().str[:3]

def bucket_age(a):
    if pd.isna(a): return np.nan
    a = int(a)
    if a < 25: return 'under_25'
    if a < 35: return '25_34'
    if a < 45: return '35_44'
    if a < 55: return '45_54'
    if a < 65: return '55_64'
    return '65_plus'

if 'age' in df.columns:
    df['age_band'] = df['age'].apply(bucket_age)

for col in ['housing', 'loan', 'default']:
    if col in df.columns:
        df[col] = df[col].replace({'yes': 'yes', 'no': 'no'})
        df[f'has_{col}'] = np.where(df[col].eq('yes'), 1, 0)

In [10]:
#QA checks Null counts (top 10)
df.isna().sum().sort_values(ascending=False).head(10)

if 'converted' in df.columns:
    print("\nOverall conversion rate:",
          df['converted'].mean()*100, 2)

if {'contact_month','converted'}.issubset(df.columns):
    print("\nConversion by month (first 5 rows):")
    print(df.groupby('contact_month')['converted'].mean().round(4)*100)



Overall conversion rate: 47.38398136534671 2

Conversion by month (first 5 rows):
contact_month
2014-01    41.28
2014-02    56.83
2014-03    89.86
2014-04    62.51
2014-05    32.75
2014-06    44.68
2014-07    41.41
2014-08    45.29
2014-09    84.33
2014-10    82.40
2014-11    42.74
2014-12    90.91
Name: converted, dtype: float64


In [11]:
df.to_csv("bank_cleaned_operational.csv", index=False)

In [12]:
df.to_excel("bank_cleaned_operational.xlsx", index=False)